In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.by import By

In [ ]:
#fonction qui recupere les tags avec comme attribut data-ar-restaurant-id
def restaurants_id (tag):
    return tag.has_attr('data-ar-restaurant-id')

In [ ]:
#fonction qui recupere les tags avec comme attribut data-product-id
def plats_id (tag):
    return tag.has_attr('data-product-id')

In [ ]:
#Fonction qui recupere les plats d'un restaurant donné et les ajoute à une liste
def recuperer_liste_plats(link, id_restaurant, liste_plats):
    req_carte = requests.get(link)

    page_carte = req_carte.text
    soup_carte = BeautifulSoup(page_carte, 'html.parser')

    plats = soup_carte.find_all(plats_id)
    
    for plat in plats:
        soup_p = BeautifulSoup(str(plat) , 'html.parser')
        #print (soup_p.prettify())
        
        #id
        id_plat = soup_p.find("li")["data-product-id"]

        #On vérifie que le plat ajouté n'est pas deja dans la liste
        isInList = False
        for p in liste_plats:
            if p["id"]==id_plat:
                isInList = True

        if not isInList:
            #nom du plat
            name = soup_p.find("h4" , {"class" : "name"}).find("span").text

            #logo
            logo = ""
            try:
                logo = soup_p.find("img" )['src']
            except:
                pass
            
            #prix
            prix = soup_p.find("div" , {"class" : "price"}).text
            prix = " ".join(prix.split())

            #description
            description = soup_p.find("div" , {"class" : "description"}).text

            objet = {
                "id_restaurant" : id_restaurant,
                "id" : id_plat,
                "name" : name,
                "prix" : prix,
                "descrption" : description,
                "logo" : logo
            }
            #print(objet["prix"])

            liste_plats.append(objet)

In [ ]:
#Fonction qui recupere les donnees des restaurants sur une page alloresto et les ajoute à une liste
def recuperer_liste_restaurants(codeSource, liste_restaurants, liste_plats):
    #req = requests.get(link)
    #page = req.text
    soup = BeautifulSoup(codeSource, 'html.parser')
    restaurants = soup.find_all(restaurants_id)
    i=0

    for restaurant in restaurants:
        restaurant = str(restaurant) 
        soup_r = BeautifulSoup(restaurant, 'html.parser')

        #link
        link = soup_r.find("h5" , {"class" : "name"}).find("a")["href"]

        #On vérifie que le restaurant n'est pas déjà dans la liste
        isInList = False
        for r in liste_restaurants:
            if r["link"]==link:
                isInList = True

        if not isInList:

            #id
            id_restaurant = i
            i = i + 1

            #name
            name = soup_r.find("h5" , {"class" : "name"}).text.replace("\n","")

            #logo
            logo = ""
            try:
                logo = soup_r.find("img" )['src']
            except:
                pass
            
            #address
            address = soup_r.find("div" , {"class" : "address"}).text
            address = " ".join(address.split())

            #specialties
            specialties = [x.text for x in soup_r.find("div" , {"class" : "cuisines"}).find_all("li")]

            #work time 
            #TODO: A RETRAVAILLER
            work_time = None

            #minimum_order
            temp = soup_r.find("li", {"class" : "minimum-order"})
            if temp is not None:
                minimum_order = temp.find("strong").text
                minimum_order = " ".join(minimum_order.split())
            else : 
                minimum_order = "0€"

            #print(type(name))
            objet = {
                "id" : id_restaurant,
                "name" : name,
                "logo" : logo,
                "address" : address,
                "specialties" : specialties,
                "work_time" : work_time,
                "minimum_order" : minimum_order,
                "link" : link
            }

            liste_restaurants.append(objet)
            #Appel de la fonction ajoutant la liste des plats
            recuperer_liste_plats(link, id_restaurant, liste_plats)

    #print(liste_restaurants)

#Liste des restaurants
liste_restaurants = []
liste_plats = []
link = "https://www.alloresto.fr/restaurants-livraison-a-domicile/zone-livraison/paris-12"

recuperer_liste_restaurants(link, liste_restaurants, liste_plats)


liste_restaurants

len(liste_restaurants)

liste_plats

len(liste_plats)

In [ ]:
#TODO : appui sur le bouton pour avoir tous les restaurants

In [ ]:
#test = json.dumps(liste_plats)
#test

#Tests Selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0

# Create a new instance of the Firefox driver
driver = webdriver.Firefox()
#driver = webdriver.PhantomJS()
#driver = webdriver.Remote("http://localhost:4444/wd/hub", webdriver.DesiredCapabilities.HTMLUNIT.copy())


# go to the google home page
driver.get("http://www.google.com")

# the page is ajaxy so the title is originally this:
print (driver.title)

# find the element that's name attribute is q (the google search box)
inputElement = driver.find_element_by_name("q")

# type in the search
inputElement.send_keys("cheese!")

# submit the form (although google automatically searches now without submitting)
inputElement.submit()

try:
    # we have to wait for the page to refresh, the last thing that seems to be updated is the title
    WebDriverWait(driver, 10).until(EC.title_contains("cheese!"))

    # You should see "cheese! - Google Search"
    print(driver.title)

finally:
    driver.quit()

In [ ]:
def recuperer_page_alloresto(link):
    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox()

    # go to the page
    driver.get(link)

    inputElement = driver.find_element_by_css_selector("button.show-more")
    inputElement.click()

    sortieBoucle = False 
    #i = 0
    while sortieBoucle is False:
        try:
            # attendre que la page charge les restaurant supplémentaires
            WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'show-more')))#EC.visibility_of(inputElement))
            #print(i)
            #i = i + 1
            #On charge de nouveaux restaurants
            inputElement = driver.find_element_by_css_selector("button.show-more")
            #print(inputElement)
            inputElement.click()
        except:
            sortieBoucle = True
            #raise

    #recuperer le code source de la page
    codeSource = driver.page_source
    #On quitte le web driver
    driver.quit()
    #print ("done")
    return codeSource

test = BeautifulSoup(codeSource, 'html.parser')
print(test.prettify())

In [ ]:
liste_restaurants = []
liste_plats = []
link="https://www.alloresto.fr/restaurants-livraison-a-domicile/zone-livraison/paris-12"
codeSource = recuperer_page_alloresto(link)
recuperer_liste_restaurants(codeSource, liste_restaurants, liste_plats)

In [ ]:
liste_restaurants

In [ ]:
len(liste_restaurants)

In [ ]:
liste_plats

In [ ]:
len(liste_plats)